In [1]:
import cv2 #opencv读取的格式是BGR
import os
import numpy as np
import matplotlib.pyplot as plt#Matplotlib是RGB
import time
%matplotlib inline 

In [2]:
def cv_show(name,img):
    cv2.imshow(name,img) 
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [3]:
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    elif width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    elif height is None:
        r = width / float(w)
        dim = (width, int(h * r))
    else :
        dim = (width,height)
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

In [4]:
def get_img(picture_path):
    img = cv2.imread(picture_path,0) #0表示灰度图
    img = resize(img, height = 500)
    img = cv2.medianBlur(img, 5)
    img = img[:int(img.shape[0]*0.5),:]
    return img

In [5]:
def get_edge(img):
    edged = cv2.Canny(img, 100, 200)
    cnts,a = cv2.findContours(edged, cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    return cnts

In [6]:
def Rotate90(img):
    trans_img = cv2.transpose(img)
    new_img = cv2.flip(trans_img, 1)
    return new_img

In [7]:
def img_rotate_90(warped):
    if warped.shape[0]>warped.shape[1]:
        image = Rotate90(warped)
        #cv_show('img',image2)
        return image

In [8]:
def img_rotation(img,angle):
    (h, w) = img.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h),borderValue=(0,0,0))
    #cv_show('img',rotated)
    return rotated , M

In [9]:
def get_row_changes(row):
    changes = 0
    row_len = len(row)
    for i in range(1,row_len):
        if row[i] != row[i-1]:
            changes = changes + 1
    return changes

In [10]:
def get_line_rotation(num_str,filename,img):
    for threshold in range(50,29,-5):#图片尺寸500时候，50 30
            for angle in range(0,70):
                img_,m = img_rotation(img.copy(),angle)
                #cv_show('img',img_)
                row,col = img_.shape
                for i in range(row-1,0,-1):
                    #print(i,angle,get_row_changes(img_[i])) 500时候 i 100
                    if get_row_changes(img_[i])<10  and i>threshold and i <100:  #是否是0矩阵
                        return i,angle,m,img_
            for angle in range(0,-70,-1):
                #cv_show('img',img_)
                #print(angle)
                img_,m = img_rotation(img.copy(),angle)
                row = img_.shape[0]
                for i in range(row-1,0,-1):
                    if get_row_changes(img_[i])<10  and i>threshold and i <100:  #是否是0矩阵
                        return i,angle,m,img_          
            #print('no angle,threshold is',threshold)
            print(num_str,filename,'is adjusting parameters and rematching')
    return img.shap[0]-3,0,0,img

In [11]:
def get_down_line(img):#得到有效区域底端的界限
    row = img.shape[0]
    for i in range(row-10,-1,-1):#从下向上扫描
        if get_row_changes(img[i]) < 5:  #如果变化次数小于5
            for j in range(10,-1,-1): #返回一个稍小的下界值，有效区域足够小
                if i-j>=0 and get_row_changes(img[i-j]) < 3: #线性搜索
                    #print("down_line",i-j)
                    return i - j + 1
                else:
                    return i
    return 0

In [12]:
def get_top_line(img): #得到有效区域上方的界限
    row = img.shape[0]
    for i in range(0,row-1,1):
        if get_row_changes(img[i]) >= 2:#变化值大于等于2则认为有效
            for j in range(5,-1,-1): #返回一个稍小的值，以保证图像完整
                if i-j>=0: #线性搜索
                    #print("top_line",i-j)
                    return i - j
            return i #以防万一

In [13]:
def get_img_origion_region(img,angle,top_row1,down_row,top_row2):
    img = img[top_row1:,:]
    img,m = img_o_rotation(img,angle)
    img = img[:down_row,:]
    img = img[top_row2:,:]
    return img
    

In [14]:
def get_points_stats(stat):
    x,y,w,h,s = stat# y方向是h
    left_top = [x,y]
    right_top = [x,y+h]
    left_bottom = [x+w,y]
    right_bottom = [x+w,y+h]
    #width =  np.int0(w)
    #height =  np.int0(h)
    return [left_top,right_top,right_bottom,left_bottom,w,h]

In [15]:
def four_point_transform(image,stat):
    # 获取输入坐标点
    (tl, tr, br, bl, w, h) = get_points_stats(stat)
    points = np.array([(tl, tr, br, bl)],dtype = "float32")
    # 变换后对应坐标位置
    dst =np.array([
        [0, 0],
        [0, h],
        [w, h],
        [w,0]
        ], dtype = "float32")
         
    # 计算变换矩阵
    #print(points)
    M = cv2.getPerspectiveTransform(points, dst)
    warped = cv2.warpPerspective(image, M, (w, h))

    # 返回变换后结果
    return warped

In [16]:
def get_char_region(img,stats,filename,outfile_name):
    region = []
    for (i,stat) in enumerate(stats):
        x,y,w,h,s = stat                    #y方向是h
        (min_w_h, max_w_h) = (np.min((w,h)), np.max((w,h)))
        if w*h<1000 and w*h >200 and max_w_h/min_w_h<4.3 and max_w_h/min_w_h >=1.2 :  #500 的时候是 500 200
            char = four_point_transform(img.copy(),stat)
        #cv2.imwrite('./'+outfile_name+'/'+filename+'___'+str(i) +'.jpg', char)
        #print(j)
            region.append([x,char]) #左上角横坐标，字符区域
    region = sorted(region, key=lambda x:x[0]) #区域从左到右排序
    return region

In [17]:
def single_char_match(char,modelpath):#返回某一图像对于某一数字模板的平均匹配值
    score = []
    sum = 0
    for filename in os.listdir(modelpath):
        #print(filename) 
        model = cv2.imread(modelpath+'/'+filename,0)
        #cv_show('a',model)
        model = resize(model,height=char.shape[0],width=char.shape[1])
        result = cv2.matchTemplate(char, model, cv2.TM_CCOEFF_NORMED)
        #print(result) 
        score.append(result)
    for i in score:
        sum = sum+i
    return sum/len(score)

In [18]:
def get_match_prediction(char,model_path_list,model_list):
    max_probility = 0     #ISBNX 0-9 * 顺序
    index = 0
    for (i,modelpath) in enumerate(model_path_list):
        result = single_char_match(char,modelpath)
        #print(modelpath,result)
        if result >= max_probility:
            max_probility = result
            index = i
    #print(model_list[index],max_probility)
    if max_probility <= 0.2: #未匹配上
        return (model_list[15],-1) #返回 *所在的下标 和异常值
    else:
        return (model_list[index],max_probility)

In [19]:
def judge(e):
    if e == '0':
        return True
    elif e == '1':
        return True
    elif e == '2':
        return True
    elif e == '3':
        return True
    elif e == '4':
        return True
    elif e == '5':
        return True
    elif e == '6':
        return True
    elif e == '7':
        return True
    elif e == '8':
        return True
    elif e == '9':
        return True
    #elif e == 'x':
        #return True
    elif e == 'X':
        return True
    else:
        return False

In [20]:
def get_char_correct_number(result_list,object_list):
    tmplist = filter(judge, result_list)
    result_list = list(tmplist)
    
    tmplist1 = filter(judge, object_list)
    object_list = list(tmplist1)    
    
    correct_number = 0  #识别正确的字符
    total_correct_number = 0 #数字全部正确的数 0或1
    object_number = len(object_list)  #目标图片的有效字符数
    if  "".join(result_list) ==  "".join(object_list):
        total_correct_number = 1
    for i in result_list:
        if i in object_list:
            correct_number = correct_number + 1
            object_list.remove(i)
    return (correct_number,object_number,total_correct_number)

In [21]:
def start_process(file_pathname):
    time_start = time.time() #开始计时
    #遍历该目录下的所有图片文件
    correct_number = 0  #所有字符的数量   算准确率
    char_correct_number = 0  #字符正确数  算准确率
    total_correct_number = 0  #全部正确的个数 用来算 正确率
    result_list = []   #识别出的字符集
    object_list = []   #正确的标签
    total_number = len(os.listdir(file_pathname))
    for (i,filename) in enumerate(os.listdir(file_pathname)):
        img = get_img(file_pathname+'/'+filename)
        img_copy = img.copy()
        ret, img_binary = cv2.threshold(img, 0 , 255, cv2.THRESH_OTSU)#得到二值图
        img_binary = cv2.bitwise_not(img_binary)#二值图反色
        num_str = '('+str(i+1)+'/'+str(total_number)+')'
        print(num_str,filename,'is matching')
        top_row1 = get_top_line(img_binary)#得到有效区域上方的界限
        img_binary = img_binary[top_row1:,:]
        down_row,angle,m,img = get_line_rotation(num_str,filename,img_binary)#矫正图像，并返回旋转矩阵
        img = img[:down_row,:]
        top_row2 = get_top_line(img)#得到有效区域上方的界限
        img = img[top_row2:,:]
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)
        char_region_list = get_char_region(img,stats,filename,'text1')
        if len(char_region_list) == 0: #防止反色图的情况
            img = cv2.bitwise_not(img)
            #cv_show('a',img)
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)
            char_region_list = get_char_region(img,stats,filename,'text1')
        print(num_str,'char_region_list length:',len(char_region_list))
        object_list = list(filename)
        result_list = [] 
        for (i,char_object) in  enumerate(char_region_list):
            prediction_char ,probility= get_match_prediction(char_object[1],model_path_list,model_list)
            result_list.append(prediction_char)
        print(num_str,'原图像:',"".join(object_list),'识别结果:',"".join(result_list))
        n1,n2,n3 = get_char_correct_number(result_list,object_list)
        char_correct_number = char_correct_number +n1
        correct_number = correct_number+n2
        total_correct_number = total_correct_number + n3
    print('char_correct_number:',char_correct_number,'correct_number:',correct_number)
    print('正确率:',total_correct_number/total_number,'准确率:',char_correct_number/correct_number)
    #print(total_correct_number,total_number)
    time_end = time.time()    #结束计时
    time_c= time_end - time_start   #运行所花时间
    print('time cost', time_c/60, '分钟')

In [22]:
model_0_path = './model_real/model_0'
model_1_path = './model_real/model_1'
model_2_path = './model_real/model_2'
model_3_path = './model_real/model_3'
model_4_path = './model_real/model_4'
model_5_path = './model_real/model_5'
model_6_path = './model_real/model_6'
model_7_path = './model_real/model_7'
model_8_path = './model_real/model_8'
model_9_path = './model_real/model_9'
model_I_path = './model_real/model_I'
model_S_path = './model_real/model_S'
model_B_path = './model_real/model_B'
model_N_path = './model_real/model_N'
model_X_path = './model_real/model_X'
model_path_list = [model_I_path,model_S_path,model_B_path,model_N_path,model_X_path,
                   model_0_path,model_1_path,model_2_path,model_3_path,model_4_path,
                   model_5_path,model_6_path,model_7_path,model_8_path,model_9_path]
model_list = ['I','S','B','N','X','0','1','2','3','4','5','6','7','8','9','*']# *代表无匹配项

In [23]:
start_process('./real_picture')

(1/100) ISBN 7-103-02208-9.jpg is matching
(1/100) char_region_list length: 14
(1/100) 原图像: ISBN 7-103-02208-9.jpg 识别结果: ISBN7103022089
(2/100) ISBN 7-300-05314-9.png is matching
(2/100) char_region_list length: 13
(2/100) 原图像: ISBN 7-300-05314-9.png 识别结果: ISBN730005349
(3/100) ISBN 7-301-08526-5.png is matching
(3/100) ISBN 7-301-08526-5.png is adjusting parameters and rematching
(3/100) ISBN 7-301-08526-5.png is adjusting parameters and rematching
(3/100) char_region_list length: 12
(3/100) 原图像: ISBN 7-301-08526-5.png 识别结果: SB7801085205
(4/100) ISBN 7-307-05433-7.png is matching
(4/100) char_region_list length: 1
(4/100) 原图像: ISBN 7-307-05433-7.png 识别结果: X
(5/100) ISBN 7-5306-3299-X.jpg is matching
(5/100) char_region_list length: 14
(5/100) 原图像: ISBN 7-5306-3299-X.jpg 识别结果: ISBN753063299X
(6/100) ISBN 7-5624-3041-1.png is matching
(6/100) char_region_list length: 14
(6/100) 原图像: ISBN 7-5624-3041-1.png 识别结果: ISBN7562430411
(7/100) ISBN 7-80151-429-7.png is matching
(7/100) char_regio

(43/100) 原图像: ISBN 978-7-5035-5305-9.jpg 识别结果: ISBN9787503553059
(44/100) ISBN 978-7-5042-1044-9.jpg is matching
(44/100) char_region_list length: 17
(44/100) 原图像: ISBN 978-7-5042-1044-9.jpg 识别结果: ISBN9787504210449
(45/100) ISBN 978-7-5043-6655-9.png is matching
(45/100) ISBN 978-7-5043-6655-9.png is adjusting parameters and rematching
(45/100) char_region_list length: 16
(45/100) 原图像: ISBN 978-7-5043-6655-9.png 识别结果: SBN9787504366559
(46/100) ISBN 978-7-5043-6838-6.png is matching
(46/100) ISBN 978-7-5043-6838-6.png is adjusting parameters and rematching
(46/100) ISBN 978-7-5043-6838-6.png is adjusting parameters and rematching
(46/100) ISBN 978-7-5043-6838-6.png is adjusting parameters and rematching
(46/100) char_region_list length: 17
(46/100) 原图像: ISBN 978-7-5043-6838-6.png 识别结果: ISBN9787504368386
(47/100) ISBN 978-7-5044-9264-7.png is matching
(47/100) ISBN 978-7-5044-9264-7.png is adjusting parameters and rematching
(47/100) char_region_list length: 17
(47/100) 原图像: ISBN 978-7-5

(79/100) ISBN 978-7-5354-9165-7.png is adjusting parameters and rematching
(79/100) ISBN 978-7-5354-9165-7.png is adjusting parameters and rematching
(79/100) char_region_list length: 16
(79/100) 原图像: ISBN 978-7-5354-9165-7.png 识别结果: ISBN978755549657
(80/100) ISBN 978-7-5378-4597-7.jpg is matching
(80/100) char_region_list length: 17
(80/100) 原图像: ISBN 978-7-5378-4597-7.jpg 识别结果: ISBN9787537845977
(81/100) ISBN 978-7-5426-4200-4.png is matching
(81/100) char_region_list length: 16
(81/100) 原图像: ISBN 978-7-5426-4200-4.png 识别结果: SBN9787542642004
(82/100) ISBN 978-7-5439-6801-1.jpg is matching
(82/100) ISBN 978-7-5439-6801-1.jpg is adjusting parameters and rematching
(82/100) char_region_list length: 17
(82/100) 原图像: ISBN 978-7-5439-6801-1.jpg 识别结果: ISBN9787543968011
(83/100) ISBN 978-7-5474-2438-4.jpg is matching
(83/100) ISBN 978-7-5474-2438-4.jpg is adjusting parameters and rematching
(83/100) char_region_list length: 14
(83/100) 原图像: ISBN 978-7-5474-2438-4.jpg 识别结果: ISBN9787547N93
(84